In [ ]:
import streamlit as st
import pickle
import re

# Load the trained model
try:
    with open("ticket_classifier.pkl", "rb") as f:
        model = pickle.load(f)
except FileNotFoundError:
    st.error("Model not found! Please run train_model.py first.")
    st.stop()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

# App Layout
st.title("🎫 SmartTicket AI")
st.subheader("IT Support Ticket Categorization")

# Sidebar
st.sidebar.header("About")
st.sidebar.info(
    "This NLP tool classifies IT support tickets into categories "
    "(Hardware, Software, Network, Access) to automate helpdesk routing."
)

# Input Area
user_text = st.text_area("Enter Ticket Description:", height=100,
                         placeholder="E.g., My internet is not working and I can't login.")

if st.button("Classify Ticket"):
    if user_text:
        # Preprocess
        cleaned_text = clean_text(user_text)

        # Predict
        prediction = model.predict([cleaned_text])[0]
        prediction_prob = model.predict_proba([cleaned_text])
        confidence = prediction_prob.max()

        # Display Results
        st.success(f"**Category:** {prediction}")
        st.info(f"**Confidence Score:** {confidence:.2%}")

        # Logic for Urgency (Rule-based simple check)
        urgent_keywords = ['urgent', 'asap', 'critical', 'immediately']
        if any(word in cleaned_text for word in urgent_keywords):
            st.warning("⚠️ Flagged as URGENT")

    else:
        st.warning("Please enter some text.")